#### Imports

In [ ]:

import json
import math
import random
import time
import queue
import shutil 
import os
import sys
import math
from enum import Enum
from datetime import datetime

import carla
import numpy as np
import urdf_parser_py.urdf as urdf
from scipy.spatial.transform import Rotation   
import matplotlib.pyplot as plt
from matplotlib import cm
import open3d as o3d
import cv2
import pygame

from classes.CARLASemantics_v0_9_15 import SemanticColors, SemanticTags

In [ ]:
# Set the CARLA Python API path
CURRENT_WORKING_DIR = os.getcwd()
CARLA_PYTHON_API_PATH = os.path.join(CURRENT_WORKING_DIR, "CARLASimulator", "PythonAPI", "carla")
if CARLA_PYTHON_API_PATH not in sys.path:
    sys.path.append(CARLA_PYTHON_API_PATH)

# Import the BasicAgent
try:
    from agents.navigation.basic_agent import BasicAgent
    from agents.navigation.behavior_agent import BehaviorAgent
except ImportError as error:
    raise ImportError(f"FATAL ERROR: Unable to import CARLA autonomous driving agent BasicAgent due to missing PythonAPI. The API is included in the simulator installation package (not included with 'import carla'). Setup the CARLA simulator repository and add the correct PythonAPI path above. ({error})")

#### Connect to CARLA server

In [ ]:
client = carla.Client('localhost', 2000)
client.set_timeout(100)
world = client.get_world()
map = world.get_map()

blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [ ]:
def reload_world():
    global world, map, blueprint_library, spawn_points, traffic_manager
    world = client.reload_world()
    map = world.get_map()
    blueprint_library = world.get_blueprint_library()
    spawn_points = map.get_spawn_points()
    traffic_manager = client.get_trafficmanager()

def load_world(map_name="Town01", timeout=10.0):
    global world, map, blueprint_library, spawn_points, traffic_manager
    client.set_timeout(timeout)
    world = client.load_world(map_name)
    map = world.get_map()
    blueprint_library = world.get_blueprint_library()
    spawn_points = map.get_spawn_points()
    traffic_manager = client.get_trafficmanager()

load_world(map_name="Town01")

In [ ]:
class CarlaSyncMode(object):
    """
        with CarlaSyncMode(world, sensors) as sync_mode:
            while True:
                data = sync_mode.tick(timeout=1.0)

    """

    def __init__(self, world, sensors, **kwargs):
        self.world = world
        self.sensors = sensors
        self.frame = None
        self.delta_seconds = 1.0 / kwargs.get('fps', 10)
        self._queues = []
        self._settings = None

    def __enter__(self):
        self._settings = self.world.get_settings()
        self.frame = self.world.apply_settings(carla.WorldSettings(
            no_rendering_mode=False,
            synchronous_mode=True,
            fixed_delta_seconds=self.delta_seconds,
            ))

        def make_queue(register_event):
            q = queue.Queue()
            register_event(q.put)
            self._queues.append(q)

        make_queue(self.world.on_tick)
        for sensor in self.sensors:
            make_queue(sensor.listen)
        return self

    def tick(self, timeout=300):
        self.frame = self.world.tick()
        data = [self._retrieve_data(q, timeout) for q in self._queues]
        assert all(x.frame == self.frame for x in data)
        return data

    def __exit__(self, *args, **kwargs):
        self.world.apply_settings(self._settings)

    def _retrieve_data(self, sensor_queue, timeout):
        while True:
            data = sensor_queue.get(timeout=timeout)
            if data.frame == self.frame:
                return data

#### CARLA transformation functions

In [ ]:
def matrix_to_transform(matrix):
    # Ensure matrix is a NumPy array
    if not isinstance(matrix, np.ndarray):
        matrix = np.array(matrix)

    # Extract translation
    location = carla.Location(x=matrix[0, 3], y=(-matrix[1, 3]), z=matrix[2, 3])

    roll, pitch, yaw = Rotation.from_matrix(matrix[:3, :3]).as_euler('xyz', degrees=True)
    rotation = carla.Rotation(pitch=(-pitch), yaw=(-yaw), roll=roll)
    
    # Create and return carla.Transform
    return carla.Transform(location, rotation)


def build_transform_matrix(rotation, translation):
    m = np.eye(4)
    m[:3, :3] = rotation
    m[:3, 3] = translation
    return m


def rotation_matrix(axis, angle):
    """
    Create a rotation matrix for a given axis and angle.
    """
    if axis == 'x':
        return np.array([
            [1, 0, 0, 0],
            [0, np.cos(angle), -np.sin(angle), 0],
            [0, np.sin(angle), np.cos(angle), 0],
            [0, 0, 0, 1]
        ])
    elif axis == 'y':
        return np.array([
            [np.cos(angle), 0, np.sin(angle), 0],
            [0, 1, 0, 0],
            [-np.sin(angle), 0, np.cos(angle), 0],
            [0, 0, 0, 1]
        ])
    elif axis == 'z':
        return np.array([
            [np.cos(angle), -np.sin(angle), 0, 0],
            [np.sin(angle), np.cos(angle), 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

def reflection_matrix():
    """
    Create a reflection matrix to flip the Y-axis.
    """
    return np.array([
        [1, 0, 0, 0],
        [0, -1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def transform_to_carla(sensor_type, transformation):
    """
    Convert sensor transformation to CARLA format.
    """
    sensor_type = sensor_type.lower().strip()
    if sensor_type in ['camera', 'sensor.camera', 'sensor.camera.rgb', 'sensor.camera.semantic_segmentation', 'sensor.camera.depth']:
        rotation1 = rotation_matrix('z', np.pi / 2)
        rotation2 = rotation_matrix('y', -np.pi / 2)
        rotation = np.dot(rotation1, rotation2)
    elif sensor_type in ['lidar', 'sensor.lidar', 'sensor.lidar.ray_cast', 'sensor.lidar.ray_cast_semantic']:
        rotation = rotation_matrix('z', np.pi / 2)
    elif sensor_type in ['radar', 'sensor.other.radar']:
        rotation = np.eye(4)
    else:
        raise ValueError("Unknown sensor type")
    tf = np.dot(transformation, rotation)
    return tf

def transform_from_carla(sensor_type, transformation):
    """
    Convert sensor transformation from CARLA format.
    """
    sensor_type = sensor_type.lower().strip()
    if sensor_type in ['camera', 'sensor.camera', 'sensor.camera.rgb', 'sensor.camera.semantic_segmentation', 'sensor.camera.depth']:
        rotation1 = rotation_matrix('z', -np.pi / 2)
        rotation2 = rotation_matrix('y', np.pi / 2)
        rotation = np.dot(rotation2, rotation1)
    elif sensor_type in ['lidar', 'sensor.lidar', 'sensor.lidar.ray_cast', 'sensor.lidar.ray_cast_semantic']:  
        rotation = rotation_matrix('z', -np.pi / 2)
    elif sensor_type in ['radar', 'sensor.other.radar']:
        rotation = np.eye(4)
    else:
        raise ValueError("Unknown sensor type")
    tf = np.dot(transformation, rotation)
    return tf

### Run path validation in CARLA

In [ ]:
planned_path_json = dict()
with open('/home/leppsalu/Desktop/Github/CARLA-vehicle-simulation/src/config/town2.path.json') as path_file:
    planned_path_json = json.load(path_file)
planned_path = planned_path_json["route"]
planned_path = planned_path[0:2]

def get_agent_path(coordinates):
    path = []
    for x, y, z in coordinates:
        location = carla.Location(x=x, y=y, z=z)
        waypoint = map.get_waypoint(
            location,
            project_to_road=True, 
            lane_type=(carla.LaneType.Driving))
        waypoint_location = waypoint.transform.location
        path.append(waypoint_location)
    return path

In [ ]:
def validate_path():
    reload_world()

    cv2.namedWindow("Press Q to stop simulation")
    cv2.imshow("Press Q to stop simulation", np.zeros((1,1)))

    blueprint_name = "vehicle.micro.microlino"
    blueprint = blueprint_library.find(blueprint_name)
    transform = spawn_points[0]
    validation_vehicle = world.spawn_actor(blueprint, transform)

    blueprint = blueprint_library.find("sensor.camera.rgb")
    blueprint.set_attribute('image_size_x', str(720))
    blueprint.set_attribute('image_size_y', str(480))

    transform_matrix = carla.Transform(carla.Location(x=1.7, y=0.0, z=1.5), carla.Rotation(roll=0, pitch=0, yaw=0)).get_matrix()
    transform = matrix_to_transform(transform_matrix)
    sensor = world.spawn_actor(blueprint, transform, attach_to=validation_vehicle)
    validation_vehicle_control_agent = BasicAgent(validation_vehicle)#, target_speed=15)
    agent_path = get_agent_path(planned_path)
    validation_vehicle_control_agent.set_destination(agent_path.pop(0))

    agent_path_completed = False
    try:
        with CarlaSyncMode(world, [sensor], fps=10) as sync_mode:
            print("Driving to the start of the path...")
            while validation_vehicle_control_agent.done() is False:
                if (cv2.waitKey(1) == ord('q')):
                    break
                simulated_results = sync_mode.tick()[1:]
                cam_sensor = simulated_results[0]
                cam_sensor_image =  np.reshape(np.copy(cam_sensor.raw_data), (cam_sensor.height, cam_sensor.width, 4))
                cv2.imshow("CAM_FRONT", cam_sensor_image)

                validation_vehicle.apply_control(validation_vehicle_control_agent.run_step())
            print("Arrived at the start of the path!")

            print("Driving on path...")
            actual_path = []
            while True:
                if (cv2.waitKey(1) == ord('q')):
                    break

                simulated_results = sync_mode.tick()[1:]
                cam_sensor = simulated_results[0]
                cam_sensor_image =  np.reshape(np.copy(cam_sensor.raw_data), (cam_sensor.height, cam_sensor.width, 4))
                cv2.imshow("CAM_FRONT", cam_sensor_image)

                location = validation_vehicle.get_location()
                actual_path.append((location.x, location.y, location.z))
                
                if validation_vehicle_control_agent.done():
                    print(f"{datetime.now()} Checkpoint reached. Validation vehicle has reached {len(planned_path) - len(agent_path)}/{len(planned_path)} planned path points.")
                    if (agent_path == []):
                        agent_path_completed = True
                        break
                    validation_vehicle_control_agent.set_destination(agent_path.pop(0))
                validation_vehicle.apply_control(validation_vehicle_control_agent.run_step())
            print("Drive finished!")
    except RuntimeError as error:
        print(error)
    finally:
        cv2.destroyAllWindows()
        validation_vehicle.destroy()

    assert agent_path_completed, "Validation failed. The vehicle could not complete the planned path."

    return planned_path, actual_path


SKIP_VALIDATION = True
validation_results = dict()
if not SKIP_VALIDATION:
    planned_path, actual_path = validate_path()
    validation_results["planned_path"] = planned_path
    validation_results["actual_path"] = actual_path

In [ ]:
def filter_path(coords, min_distance):
    def distance(point1, point2):
        return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    filtered_coords = []
    for i in range(len(coords)):
        if (i == 0) or (i == (len(coords)-1)) or distance(filtered_coords[-1], coords[i]) >= min_distance:
            filtered_coords.append(coords[i])
    return filtered_coords

def visualize_paths(planned_path, actual_path):
    # Initialize Pygame
    pygame.init()

    # Constants
    WIDTH, HEIGHT = 800, 600
    BACKGROUND_COLOR = (255, 255, 255)
    NODE_COLOR = (0, 0, 255)
    EDGE_COLOR = (0, 0, 0)  # For the planned coordinates
    PLANNED_NODE_COLOR = (0, 255, 0)
    PLANNED_EDGE_COLOR = (0, 200, 0)
    NODE_RADIUS = 5
    ARROW_SIZE = 10
    FONT_COLOR = (0, 0, 0)

    # Create the screen
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Validation path summary")

    # Zoom and pan variables
    scale = 1.0
    offset_x, offset_y = 0, 0
    zoom_factor = 1.1
    dragging = False
    last_mouse_pos = (0, 0)

    # Checkbox states
    show_actual_path = True
    show_planned_path = True

    # Slider values
    actual_path_distance_filter = 10.0
    planned_path_distance_filter = 10.0

    def draw_arrow(surface, start, end, color):
        pygame.draw.line(surface, color, start, end, 2)
        
        angle = math.atan2(end[1] - start[1], end[0] - start[0])
        left_angle = angle + math.pi / 6
        right_angle = angle - math.pi / 6

        left_point = (end[0] - ARROW_SIZE * math.cos(left_angle),
                      end[1] - ARROW_SIZE * math.sin(left_angle))
        right_point = (end[0] - ARROW_SIZE * math.cos(right_angle),
                       end[1] - ARROW_SIZE * math.sin(right_angle))

        pygame.draw.polygon(surface, color, [end, left_point, right_point])

    def draw_nodes_and_edges(coords, node_color, edge_color):
        for i, (x, y, z) in enumerate(coords):
            scaled_x = (x + offset_x) * scale
            scaled_y = (y + offset_y) * scale
            
            pygame.draw.circle(screen, node_color, (int(scaled_x), int(scaled_y)), NODE_RADIUS)
            
            font = pygame.font.Font(None, 24)
            text_surface = font.render(str(i), True, FONT_COLOR)
            screen.blit(text_surface, (scaled_x + 10, scaled_y - 10))
            
            if i < len(coords) - 1:
                next_x, next_y, _ = coords[i + 1]
                next_scaled_x = (next_x + offset_x) * scale
                next_scaled_y = (next_y + offset_y) * scale
                draw_arrow(screen, (int(scaled_x), int(scaled_y)), (int(next_scaled_x), int(next_scaled_y)), edge_color)

    def draw_checkbox(x, y, label, checked):
        pygame.draw.rect(screen, (0, 0, 0), (x, y, 20, 20), 2)
        if checked:
            pygame.draw.rect(screen, (0, 255, 0), (x + 2, y + 2, 16, 16))

        font = pygame.font.Font(None, 24)
        text_surface = font.render(label, True, FONT_COLOR)
        screen.blit(text_surface, (x + 30, y))

    def draw_slider(x, y, value):
        pygame.draw.rect(screen, (200, 200, 200), (x, y, 200, 20))  # Slider background
        pygame.draw.rect(screen, (100, 100, 100), (x + int(value), y, 10, 20))  # Slider handle

    # Main loop
    clock = pygame.time.Clock()
    running = True
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
            
            # Zoom with mouse scroll wheel
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4:  # Scroll up
                    scale *= zoom_factor
                elif event.button == 5:  # Scroll down
                    scale /= zoom_factor
                elif event.button == 1:  # Left mouse button
                    dragging = True
                    last_mouse_pos = pygame.mouse.get_pos()

            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:  # Left mouse button
                    dragging = False
                # Click on checkboxes
                mouse_pos = pygame.mouse.get_pos()
                if 10 <= mouse_pos[0] <= 30 and 10 <= mouse_pos[1] <= 30:
                    show_actual_path = not show_actual_path
                if 10 <= mouse_pos[0] <= 30 and 90 <= mouse_pos[1] <= 110:
                    show_planned_path = not show_planned_path

            elif event.type == pygame.MOUSEMOTION:
                mouse_on_slider = False
                if dragging:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    if (10 <= mouse_x <= 210) and (40 <= mouse_y <= 80):  # Adjust value within slider range
                        actual_path_distance_filter = (mouse_x - 10) 
                        mouse_on_slider = True
                    if (10 <= mouse_x <= 210) and (120 <= mouse_y <= 160):
                        planned_path_distance_filter = (mouse_x - 10)
                        mouse_on_slider = True
                    
        # Drag to pan
        if dragging:
            mouse_x, mouse_y = pygame.mouse.get_pos()
            if not mouse_on_slider:
                offset_x -= (last_mouse_pos[0] - mouse_x) / scale
                offset_y -= (last_mouse_pos[1] - mouse_y) / scale
            last_mouse_pos = (mouse_x, mouse_y)

        # Clear the screen
        screen.fill(BACKGROUND_COLOR)

        draw_checkbox(10, 10, "Show Actual Path", show_actual_path)
        draw_slider(10, 50, actual_path_distance_filter)  # Scale for display
        

        draw_checkbox(10, 90, "Show Planned Path", show_planned_path)
        draw_slider(10, 130, planned_path_distance_filter)

        
        if show_actual_path:
            filtered_actual_path = filter_path(actual_path, actual_path_distance_filter) # Filtered path based on slider values
            draw_nodes_and_edges(filtered_actual_path, NODE_COLOR, EDGE_COLOR)
        if show_planned_path:
            filtered_planned_path = filter_path(planned_path, planned_path_distance_filter) # Filtered path based on slider values
            draw_nodes_and_edges(filtered_planned_path, PLANNED_NODE_COLOR, PLANNED_EDGE_COLOR)

        # Update the display
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()

if not SKIP_VALIDATION:
    visualize_paths(validation_results["planned_path"], validation_results["actual_path"])

### Prepare CARLA world for full simulation

#### Add ego vehicle (with intrinsics and extrinsics from external configuration)

In [ ]:
class URDFParser:
    def __init__(self, urdf_file):
        self.urdf_file = urdf_file
        self.robot = urdf.URDF.from_xml_file(urdf_file)
        self.root = self.robot.get_root()

    def compute_chain_transform(self, chain):
        transform = np.eye(4)
        
        for joint in chain:
            if joint not in self.robot.joint_map:
                continue
            
            joint_info = self.robot.joint_map[joint]
            rpy = joint_info.origin.rpy
            xyz = joint_info.origin.xyz
            rotation = Rotation.from_euler('xyz', rpy).as_matrix()
            translation = np.array(xyz)
            T = build_transform_matrix(rotation, translation)
            transform = np.dot(transform, T)
        
        return transform

    def get_T_from_to(self, start_frame, end_frame):
        chain_1 = self.robot.get_chain(self.root, start_frame)
        chain_2 = self.robot.get_chain(self.root, end_frame)
        T1 = self.compute_chain_transform(chain_1)
        T2 = self.compute_chain_transform(chain_2)
        return np.dot(np.linalg.inv(T1), T2)

In [ ]:
reload_world()

blueprint_name = "vehicle.micro.microlino"
blueprint = blueprint_library.find(blueprint_name)
blueprint.set_attribute('role_name','ego')
transform = spawn_points[0]
vehicle = world.spawn_actor(blueprint, transform)

sensor_names = []
sensor_types = []
sensors = []

axes = []

extrinsics = URDFParser('config/carla_extrinsics.urdf')
intrinsics = dict()
with open('config/carla_intrinsics.json') as intrinsics_file:
    intrinsics = json.load(intrinsics_file)

for sensor_configuration in extrinsics.robot.links:
    sensor_name = sensor_configuration.name

    if "CAM_" in sensor_name:
        blueprint_name = "sensor.camera.rgb"
        sensor_type = blueprint_name
        blueprint = blueprint_library.find(blueprint_name)
        sensor_intrinsics = intrinsics.get(sensor_name, dict())

        def calculate_fov(focal_length, image_width):
            fov_radians = 2 * np.arctan(image_width / (2 * focal_length))
            fov_degrees = np.degrees(fov_radians)
            return fov_degrees
        image_width = str(sensor_intrinsics.get("w", 1600))
        blueprint.set_attribute('image_size_x', image_width)
        image_height = str(sensor_intrinsics.get("h", 900))
        blueprint.set_attribute('image_size_y', image_height)
        focal_distance = float(sensor_intrinsics.get("fl"))
        field_of_view = str(calculate_fov(focal_distance, float(image_width)))
        blueprint.set_attribute('fov', field_of_view)
        
        transform_matrix = extrinsics.compute_chain_transform(extrinsics.robot.get_chain(extrinsics.root, sensor_name))
        transform_matrix = transform_to_carla("camera", transform_matrix)
        transform = matrix_to_transform(transform_matrix)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)
        sensors.append(sensor)
        sensor_types.append(sensor_type)
        sensor_names.append(sensor_name)

        ## Spawn additional DEPTH and SEMANTIC cams
        blueprint_name = "sensor.camera.depth"
        sensor_type = blueprint_name
        blueprint = blueprint_library.find(blueprint_name)
        blueprint.set_attribute('image_size_x', image_width)
        blueprint.set_attribute('image_size_y', image_height)
        blueprint.set_attribute('fov', field_of_view)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)
        sensors.append(sensor)
        sensor_types.append(sensor_type)
        sensor_names.append(f"DEPTH_{sensor_name}")

        blueprint_name = "sensor.camera.semantic_segmentation"
        sensor_type = blueprint_name
        blueprint = blueprint_library.find(blueprint_name)
        blueprint.set_attribute('image_size_x', image_width)
        blueprint.set_attribute('image_size_y', image_height)
        blueprint.set_attribute('fov', field_of_view)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)
        sensors.append(sensor)
        sensor_types.append(sensor_type)
        sensor_names.append(f"SEMANTIC_{sensor_name}")

    elif "RADAR_" in sensor_name:
        blueprint_name = "sensor.other.radar"
        sensor_type = blueprint_name
        blueprint = blueprint_library.find(blueprint_name)     
        blueprint.set_attribute('horizontal_fov', str(30.0)) 
        blueprint.set_attribute('vertical_fov', str(30.0)) 
        blueprint.set_attribute('points_per_second', str(1e5))
        
        transform_matrix = extrinsics.get_T_from_to("center", sensor_name)
        transform_matrix = transform_to_carla('radar', transform_matrix)
        transform = matrix_to_transform(transform_matrix)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)
        sensors.append(sensor)
        sensor_types.append(sensor_type)
        sensor_names.append(sensor_name)

    elif "LIDAR_" in sensor_name:
        blueprint_name = "sensor.lidar.ray_cast_semantic"
        sensor_type = blueprint_name
        blueprint = blueprint_library.find(blueprint_name)
        blueprint.set_attribute("channels", str(64))
        blueprint.set_attribute("points_per_second", str(112000))
        blueprint.set_attribute("range", str(100))
        
        transform_matrix = extrinsics.get_T_from_to("center", sensor_name)
        transform_matrix = transform_to_carla('lidar', transform_matrix)
        transform = matrix_to_transform(transform_matrix)
        attached_to = vehicle
        sensor = world.spawn_actor(blueprint, transform, attach_to=attached_to)
        sensors.append(sensor)
        sensor_types.append(sensor_type)
        sensor_names.append(sensor_name)

In [ ]:
print(f"Number of sensors spawned: {len(sensors)}")
print(sensor_names)
print(sensor_types)

#### Add traffic (non-ego vehicles and pedestrians) to simulation

In [ ]:
def add_vehicles_to_simulation(n_vehicles: int=0):
    """
    Choos n_vehicles amount of vehicles from Carla's Blueprint Library and
    spawn them at random points on the map. Each of those vehicles are set
    on autopilot and controlled by Carla's Traffic Manager.
    
    NB! If randomly chosen spawn points collide, the vehicle is not spawned
    so there might be fewer vehicles than set by n_vehicles.

    Inputs:
        n_vehicles - amount of vehicles to spawn
    """
    if n_vehicles == 0:
        return
    
    vehicle_blueprints = blueprint_library.filter('vehicle')
    
    npcs = []
    collisions = 0
    for _ in range(n_vehicles):
        vehicle_bp = random.choice(vehicle_blueprints)
        spawn_point = random.choice(spawn_points)
        npc = world.try_spawn_actor(vehicle_bp, spawn_point)
        if npc:
            npcs.append(npc)
        else:
            collisions += 1
    
    if collisions:
        print(f"Failed to spawn {collisions}/{n_vehicles} vehicles because of collisions")
    
    port = traffic_manager.get_port()
    for npc in npcs:
        npc.set_autopilot(True, port)

def add_pedestrians_to_simulation(n_pedestrians: int=0,
                                  min_speed: float=1.0,
                                  max_speed: float=2.0) -> None:
    """
    Choose n_pedestrians amount of random pedestrians from Carla's
    Blueprint Library, attach them to AI and spawn them to random 
    positions in the simulation. Set a random location for them to 
    walk to and speed between min_speed and max_speed for them to
    walk at.

    NB! If the randomly chosen spawn points collide, a pedestrian
    is not spawned, so there might be fewer pedestrians than 
    n_pedestrians in the simulation.

    Inputs:
        n_pedestrians - number of pedestrians to spawn
        min_speed - minimum walking speed for a pedestrian
        max_speed - maximum walking speed for a pedestrian
    """
    if n_pedestrians == 0:
        return
    
    pedestrian_blueprints = blueprint_library.filter("walker.pedestrian.*")
    points = []
    for i in range(n_pedestrians):
        point = carla.Transform()
        point.location = world.get_random_location_from_navigation()
        if (point.location != None):
            points.append(point)
    batch = []

    # Create a batch of spawn commands
    for point in points:
        pedestrian_bp = random.choice(pedestrian_blueprints)
        batch.append(carla.command.SpawnActor(pedestrian_bp, point))

    # Apply the batch of commands
    results = client.apply_batch_sync(batch, True)
    walkers_list = []
    collisions = 0
    for i in range(len(results)):
        if results[i].error:
            collisions += 1
        else:
            walkers_list.append({"id": results[i].actor_id})

    if collisions:
        print(f"Failed to spawn {collisions}/{n_pedestrians} pedestrians because of collisions")

    # Create a batch of spawn commands for AI controllers
    batch = []
    walker_controller_bp = blueprint_library.find('controller.ai.walker')
    for i in range(len(walkers_list)):
        batch.append(carla.command.SpawnActor(walker_controller_bp, carla.Transform(), walkers_list[i]["id"]))

    # Apply the batch of commands
    results = client.apply_batch_sync(batch, True)
    for i in range(len(results)):
        if results[i].error:
            print(f"Spawning pedestrian AI failed: {results[i].error}")
        else:
            walkers_list[i]["con"] = results[i].actor_id

    all_ids = []
    for i in range(len(walkers_list)):
        all_ids.append(walkers_list[i]["con"])
        all_ids.append(walkers_list[i]["id"])
    all_actors = world.get_actors(all_ids)

    for i in range(0, len(all_actors), 2):
        all_actors[i].start()
        all_actors[i].go_to_location(world.get_random_location_from_navigation())
        all_actors[i].set_max_speed(random.uniform(min_speed, max_speed))

add_vehicles_to_simulation(n_vehicles=30)
add_pedestrians_to_simulation(n_pedestrians=30)

### Run full simulation in CARLA

##### Filesystem methods

In [ ]:
def delete_all_in_directory(target_directory):
    if os.path.exists(target_directory):
        for filename in os.listdir(target_directory):
            file_path = os.path.join(target_directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path) 
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')


def create_filename_from_timestamp(timestamp):
    SECONDS_TO_NANOSECONDS = 1000000000
    filename = str(math.trunc(timestamp * SECONDS_TO_NANOSECONDS))
    return filename


##### General sensor processing methods

In [ ]:
def save_sensor_position(raw_data, target_directory, sensor_type=None):
    transform_matrix = raw_data.transform.get_matrix()
    if sensor_type is not None:
        transform_matrix = transform_from_carla(sensor_type, transform_matrix)
    if not os.path.exists(target_directory):
        os.makedirs(target_directory, exist_ok=True)
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".npy"
    np.save(f"{target_directory}/{filename}", transform_matrix)

##### Camera Methods (RGB, Semantic and Depth) 

In [ ]:
def save_camera_image(raw_data, target_directory):    
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    rgb_image = np.reshape(raw_data.raw_data, (raw_data.height, raw_data.width, 4))[:, :, :3] # Remove alpha channel
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".png"
    cv2.imwrite(f"{directory}/{filename}", rgb_image)

##### Radar Methods

In [ ]:
def save_radar_readings(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    radar_points_list = []
    for measurement in raw_data:
        azi = math.degrees(measurement.azimuth)
        alt = math.degrees(measurement.altitude)
        fw_vec = carla.Vector3D(x=measurement.depth)
        carla.Transform(
            carla.Location(),
            carla.Rotation(pitch=alt,yaw=azi,roll=0)
        ).transform(fw_vec)
        radar_points_list.append([fw_vec.x, fw_vec.y, fw_vec.z])
    
    points = np.array(radar_points_list)
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".ply"
    o3d.io.write_point_cloud(f"{directory}/{filename}", point_cloud)

##### Lidar methods

In [ ]:
def save_lidar_readings(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".ply"
    raw_data.save_to_disk(f"{directory}/{filename}")

##### Semantic Lidar Methods

In [ ]:
def save_semantic_lidar_readings(raw_data, target_directory):
    directory = os.path.dirname(target_directory)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

    lidar_data = np.array([(detection.point.x, detection.point.y, detection.point.z, detection.object_tag) for detection in raw_data])
    points = lidar_data[:, :3]
    semantic_tags = lidar_data[:, 3] 
    semantic_colors_rgb = np.zeros((len(semantic_tags), 3))
    semantic_colors_rgb[:, 0] = semantic_tags
    semantic_colors_normalized = semantic_colors_rgb / 255.0
    
    # Create an Open3D point cloud object
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(semantic_colors_normalized)

    # Save the point cloud to a .ply file
    filename = create_filename_from_timestamp(raw_data.timestamp) + ".ply"
    o3d.io.write_point_cloud(f"{target_directory}/{filename}", point_cloud)

In [ ]:
SIMULATED_DATA_DIRECTORY = "./generated_data"
delete_all_in_directory(SIMULATED_DATA_DIRECTORY)

cv2.namedWindow("Press Q to stop simulation")
cv2.imshow("Press Q to stop simulation", np.zeros((1,1)))

vehicle_control_agent = BasicAgent(vehicle, target_speed=15)
agent_path = get_agent_path(planned_path)
vehicle_control_agent.set_destination(agent_path.pop(0))

try:
    with CarlaSyncMode(world, sensors, fps=10) as sync_mode:
        print("Driving to the start of the path...")
        while vehicle_control_agent.done() is False:
            if (cv2.waitKey(1) == ord('q')):
                break
            simulated_results = sync_mode.tick()[1:]
            cam_sensor = simulated_results[0]
            cam_sensor_image =  np.reshape(np.copy(cam_sensor.raw_data), (cam_sensor.height, cam_sensor.width, 4))
            cv2.imshow("CAM_FRONT", cam_sensor_image)
            
            vehicle.apply_control(vehicle_control_agent.run_step())
        print("Arrived at the start of the path!")

        print("Driving on path...")
        while True:
            simulation_results = sync_mode.tick(timeout=100.0)[1:]
            for i in range(len(simulation_results)):
                sensor = sensors[i]
                sensor_data = simulation_results[i]
                sensor_name = sensor_names[i].replace("base_link_to_", "")
                sensor_type = sensor_types[i]
                if ("camera.rgb" in sensor_type):
                    save_camera_image(sensor_data, f"generated_data/{sensor_name}/")
                elif ("camera.semantic_segmentation" in sensor_type):
                    save_camera_image(sensor_data, f"generated_data/{sensor_name}/")
                elif ("sensor.other.radar" in sensor_type):
                    save_radar_readings(sensor_data, f"generated_data/{sensor_name}/")
                elif ("camera.depth" in sensor_type):
                    save_camera_image(sensor_data, f"generated_data/{sensor_name}/") 
                elif ("sensor.lidar.ray_cast_semantic" in sensor_type):
                    save_semantic_lidar_readings(sensor_data, f"generated_data/{sensor_name}/")
                elif ("sensor.lidar" in sensor_type):
                    save_lidar_readings(sensor_data, f"generated_data/{sensor_name}/")
                save_sensor_position(sensor_data, f"generated_data/{sensor_name}/", sensor_type=sensor_type)
            
            if cv2.waitKey(1) == ord('q'):
                break

            if vehicle_control_agent.done():
                print(f"{datetime.now()} Checkpoint reached. Ego vehicle has reached {len(planned_path) - len(agent_path)}/{len(planned_path)} planned path points.")
                if len(agent_path) == 0:
                    break
                vehicle_control_agent.set_destination(agent_path.pop(0))
            vehicle.apply_control(vehicle_control_agent.run_step())

        print("Driving on path finished!")
finally:
    cv2.destroyAllWindows()
    for sensor in sensors:
        sensor.stop()